## Codificación de las variables categoricas, escalado y modelo

In [1]:
import pandas as pd 
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics
import warnings
import category_encoders as ce

In [2]:
warnings.filterwarnings('ignore')

Cargamos los datasets guardados en el notebook 2 para definir los dataframes de train y test.

In [3]:
pd_fraud_train = pd.read_csv("data/train_pd_data_preprocessing_missing_outlier.csv")
pd_fraud_test = pd.read_csv("data/test_pd_data_preprocessing_missing_outlier.csv")

Eliminamos la variable "Unnamed: 0" de ambos dataframes que se ha creado automáticamente al guardar los datasets en el notebook anterior.

In [4]:
pd_fraud_train =pd_fraud_train.drop(columns=["Unnamed: 0"])

In [5]:
pd_fraud_test=pd_fraud_test.drop(columns=["Unnamed: 0"])

Creamos tres variables: una variable que es una lista de las variables categóricas (lista_variables_categoricas), otra variable que es una lista de las variables numéricas (lista_variables_numericas) y otra variable que recoge a la variable objetivo fraud_boll (target)

In [6]:
lista_variables_categoricas = ["payment_type","employment_status","housing_status","email_is_free","phone_home_valid",
                                "phone_mobile_valid","has_other_cards","foreign_request","source","device_os","keep_alive_session", "fraud_bool"]

lista_variables_numericas = ["income", "name_email_similarity", "prev_address_months_count", "current_address_months_count", "customer_age", "days_since_request",
                            "intended_balcon_amount", "zip_count_4w", "velocity_6h", "velocity_24h", "velocity_4w", "bank_branch_count_8w", "date_of_birth_distinct_emails_4w",
                            "credit_risk_score", "bank_months_count", "proposed_credit_limit", "session_length_in_minutes", "device_distinct_emails_8w", "device_fraud_count",
                            "month"]
target = 'fraud_bool'

Convertimos las variables categóricas (lista_variables_categoricas) a tipos de datos categóricos en los DataFrames pd_fraud_train y pd_fraud_test:

In [7]:
for columna in lista_variables_categoricas:
    pd_fraud_train[columna] = pd_fraud_train[columna].astype('category')

In [8]:
for columna in lista_variables_categoricas:
    pd_fraud_test[columna] = pd_fraud_test[columna].astype('category')

Modificamos el tipo de datos de la columna "fraud_bool" (variable objetivo) en los DataFrames pd_fraud_train y pd_fraud_test a tipo de datos de cadena (str):

In [9]:
pd_fraud_train[target] = pd_fraud_train[target].astype(str)

In [10]:
pd_fraud_test[target] = pd_fraud_test[target].astype(str)

In [11]:
pd_fraud_train.columns

Index(['income', 'name_email_similarity', 'prev_address_months_count',
       'current_address_months_count', 'customer_age', 'days_since_request',
       'intended_balcon_amount', 'payment_type', 'zip_count_4w', 'velocity_6h',
       'velocity_24h', 'velocity_4w', 'bank_branch_count_8w',
       'date_of_birth_distinct_emails_4w', 'employment_status',
       'credit_risk_score', 'email_is_free', 'housing_status',
       'phone_home_valid', 'phone_mobile_valid', 'bank_months_count',
       'has_other_cards', 'proposed_credit_limit', 'foreign_request', 'source',
       'session_length_in_minutes', 'device_os', 'keep_alive_session',
       'device_distinct_emails_8w', 'device_fraud_count', 'month',
       'fraud_bool'],
      dtype='object')

In [12]:
pd_fraud_test

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,...,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month,fraud_bool
0,0.7,0.778701,-1.0,139.0,40,0.008380,-0.202636,AB,3846.0,7304.807120,...,1500.0,0,INTERNET,11.979712,windows,0,2,0,3,0
1,0.4,0.293344,-1.0,131.0,50,0.037644,5.051617,AA,1849.0,6779.956008,...,200.0,0,INTERNET,7.516187,other,0,1,0,6,0
2,0.1,0.353385,-1.0,68.0,40,0.013479,-0.535074,AD,1511.0,7134.121115,...,200.0,0,INTERNET,3.677833,other,1,1,0,3,0
3,0.1,0.104095,55.0,1.0,30,0.008555,-0.795406,AB,598.0,14307.739803,...,200.0,0,INTERNET,11.055169,linux,1,1,0,4,0
4,0.8,0.735244,-1.0,303.0,30,0.017484,50.871129,AA,1370.0,4974.196161,...,500.0,0,INTERNET,6.714066,other,0,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0.8,0.204922,32.0,32.0,20,0.010461,-1.064150,AB,1016.0,4951.200948,...,200.0,0,INTERNET,1.453849,other,1,1,0,5,0
199996,0.2,0.267594,-1.0,63.0,30,0.026145,-0.570981,AB,698.0,4825.271785,...,200.0,0,INTERNET,7.241431,linux,1,1,0,7,0
199997,0.7,0.997080,27.0,13.0,20,10.717964,47.289811,AA,2111.0,8199.098682,...,990.0,0,INTERNET,8.956753,linux,1,1,0,2,0
199998,0.9,0.054926,-1.0,45.0,40,0.025997,-1.174708,AB,635.0,6693.513234,...,200.0,0,INTERNET,10.354687,windows,1,1,0,3,0


In [13]:
pd_fraud_train.dtypes

income                               float64
name_email_similarity                float64
prev_address_months_count            float64
current_address_months_count         float64
customer_age                           int64
days_since_request                   float64
intended_balcon_amount               float64
payment_type                        category
zip_count_4w                         float64
velocity_6h                          float64
velocity_24h                         float64
velocity_4w                          float64
bank_branch_count_8w                 float64
date_of_birth_distinct_emails_4w       int64
employment_status                   category
credit_risk_score                    float64
email_is_free                       category
housing_status                      category
phone_home_valid                    category
phone_mobile_valid                  category
bank_months_count                      int64
has_other_cards                     category
proposed_c

## Codificación de la variable objetivo

In [14]:
pd_fraud_train['fraud_bool'].value_counts()

fraud_bool
0    791177
1      8823
Name: count, dtype: int64

Hemos convertido a la columna 'fraud_bool' en una columna numérica binaria donde 'Non-fraudulent Application' se representa como 0, 'Fraudulent Application' se representa como 1.

Creamos una copia del DataFrame original para evitar que se modifique y reemplazamos los valores de la columna target.

In [15]:
pd_fraud_train_str = pd_fraud_train.copy()

pd_fraud_train_str['fraud_bool'] = pd_fraud_train_str['fraud_bool'].map({"0": "No fraude", "1": "Fraude"})

pd_fraud_test_str = pd_fraud_test.copy()

pd_fraud_test_str['fraud_bool'] = pd_fraud_test_str['fraud_bool'].map({"0": "No fraude", "1": "Fraude"})



In [16]:
pd_fraud_train_str['fraud_bool'].value_counts()

fraud_bool
No fraude    791177
Fraude         8823
Name: count, dtype: int64

In [17]:
pd_fraud_test_str['fraud_bool'].value_counts()

fraud_bool
No fraude    197794
Fraude         2206
Name: count, dtype: int64

Eliminamos a la variable objetivo (fraud_bool) del conjunto de entrenamiento X_train y de test X_test puesto que va a formar parte de y_train e y_test. 

In [18]:
X_train = pd_fraud_train.drop('fraud_bool',axis=1)
X_test = pd_fraud_test.drop('fraud_bool',axis=1)
y_train = pd_fraud_train['fraud_bool']
y_test = pd_fraud_test['fraud_bool']

## Codificación del resto de variables categoricas

Creamos las variables "list_columns_cat" que contiene las columnas categóricas y "list_other" que contiene las columnas no categóricas en el DataFrame X_train. De esta forma se puede separar y tratar de manera diferente las variables categóricas y no categóricas del dataset.

In [19]:
list_columns_cat = list(X_train.select_dtypes(include=['category', 'object']).columns)
list_other = list(set(X_train.columns)-set(list_columns_cat))

In [20]:
ohe = ce.OneHotEncoder(cols=list_columns_cat)
model = ohe.fit(X_train, y_train)

Hemos codificado las variables categóricas (list_columns_cat) con one-hot-encoding y posteriormente hemos ajustado el encoder a los datos de entrenamiento. Con esto lo que hacemos es convertir las distintas categorías de una columna en: cada columna tendrá un '1' si la categoría original era esa y '0' lo contrario.

In [21]:
model

OneHotEncoder(cols=['payment_type', 'employment_status', 'email_is_free',
                    'housing_status', 'phone_home_valid', 'phone_mobile_valid',
                    'has_other_cards', 'foreign_request', 'source', 'device_os',
                    'keep_alive_session'])

"X_train_t" transforma el conjunto de entrenamiento (X_train) utilizando el modelo "model" e "X_test_t" transforma el conjunto de prueba (X_test) utilizando el mismo modelo "model".

In [22]:
X_train_t = model.transform(X_train, y_train)
X_test_t = model.transform(X_test, y_test)
# X_train_t.columns = list_columns_cat
# X_test_t.columns = list_columns_cat

Obtenemos conjuntos de datos de entrenamiento y prueba (X_train_t y X_test_t) preparados para utilizarse en un modelo de aprendizaje automático, puesto que todas las variables categóricas ya están codificadas adecuadamente.

A continuación, calculamos el número de columnas del DataFrame X_train_t, y así podemos comprobar el nuevo número de columnas que se han creado.

In [23]:
len(list(X_train_t.columns))

58

El conjunto de entrenamiento de X dispone de 31 variables 

In [24]:
X_train_t.dtypes.to_dict()

{'income': dtype('float64'),
 'name_email_similarity': dtype('float64'),
 'prev_address_months_count': dtype('float64'),
 'current_address_months_count': dtype('float64'),
 'customer_age': dtype('int64'),
 'days_since_request': dtype('float64'),
 'intended_balcon_amount': dtype('float64'),
 'payment_type_1': dtype('int64'),
 'payment_type_2': dtype('int64'),
 'payment_type_3': dtype('int64'),
 'payment_type_4': dtype('int64'),
 'payment_type_5': dtype('int64'),
 'zip_count_4w': dtype('float64'),
 'velocity_6h': dtype('float64'),
 'velocity_24h': dtype('float64'),
 'velocity_4w': dtype('float64'),
 'bank_branch_count_8w': dtype('float64'),
 'date_of_birth_distinct_emails_4w': dtype('int64'),
 'employment_status_1': dtype('int64'),
 'employment_status_2': dtype('int64'),
 'employment_status_3': dtype('int64'),
 'employment_status_4': dtype('int64'),
 'employment_status_5': dtype('int64'),
 'employment_status_6': dtype('int64'),
 'employment_status_7': dtype('int64'),
 'credit_risk_score'

Observamos los tipos de datos de "X_train_t"

# CONCLUSIONES FINALES

- Leemos los datasets creados en el notebook 2 para definir los datasets 'pd_fraud_train' y 'pd_fraud_test'.
- Eliminamos la variable "Unnamed: 0" de ambos dataframes que se ha creado automáticamente al guardar los datasets en el notebook anterior.
- Las variables "lista_variables_categoricas", "lista_variables_numericas" y "target" recogen a las variables categóricas, numéricas y la variable objetivo del dataset, respectivamente.
- Las variables categóricas son transformadas a tipo de datos categórico en pd_fraud_train y pd_fraud_test para su preprocesamiento y la variable objetivo a formato string.
- Creamos una copia del DataFrame original para evitar que se modifique y reemplazamos los valores de la columna target. A este nuevo DataFrame ('pd_fraud_train_str'), le realizamos la codificación de la variable objetivo convirtiendo a la columna 'fraud_bool'en una columna numérica binaria donde 'Non-fraudulent Application' se representa como 0, 'Fraudulent Application' se representa como 1. 
- Eliminamos a la variable objetivo del conjunto de entrenamiento X_train y de test X_test puesto que va a formar parte de y_train e y_test.
- Codificación del resto de variables categóricas: Creamos las variables "list_columns_cat" que contiene las columnas categóricas y "list_other" que contiene las columnas no categóricas en el DataFrame X_train.
- Hemos codificado las variables categóricas (list_columns_cat) con one-hot-encoding y posteriormente hemos ajustado el encoder a los datos de entrenamiento siendo los conjuntos de entramiento y test transformados: "X_train_t" y "X_test_t", respectivamente.